# Convolutional Neural net

<img src="./cnn.png">


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

In [2]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

## Weights and biases

In [3]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

## Convolutions

In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

## Max Pooling

In [5]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

## Model

In [7]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## session run : train & test


In [8]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 62443.6953 Validation Accuracy: 0.156250
Epoch  1, Batch   2 -Loss: 46680.8086 Validation Accuracy: 0.144531
Epoch  1, Batch   3 -Loss: 40237.9648 Validation Accuracy: 0.144531
Epoch  1, Batch   4 -Loss: 33485.2891 Validation Accuracy: 0.152344
Epoch  1, Batch   5 -Loss: 29509.0469 Validation Accuracy: 0.148438
Epoch  1, Batch   6 -Loss: 29819.7344 Validation Accuracy: 0.148438
Epoch  1, Batch   7 -Loss: 28467.5508 Validation Accuracy: 0.132812
Epoch  1, Batch   8 -Loss: 23109.5332 Validation Accuracy: 0.144531
Epoch  1, Batch   9 -Loss: 26493.9492 Validation Accuracy: 0.167969
Epoch  1, Batch  10 -Loss: 27937.0898 Validation Accuracy: 0.167969
Epoch  1, Batch  11 -Loss: 21500.6719 Validation Accuracy: 0.175781
Epoch  1, Batch  12 -Loss: 21997.0918 Validation Accuracy: 0.160156
Epoch  1, Batch  13 -Loss: 19019.6602 Validation Accuracy: 0.187500
Epoch  1, Batch  14 -Loss: 19458.7695 Validation Accuracy: 0.195312
Epoch  1, Batch  15 -Loss: 20570.4727 Validation

Epoch  1, Batch 122 -Loss:  3830.5740 Validation Accuracy: 0.488281
Epoch  1, Batch 123 -Loss:  4343.7188 Validation Accuracy: 0.496094
Epoch  1, Batch 124 -Loss:  4233.9746 Validation Accuracy: 0.503906
Epoch  1, Batch 125 -Loss:  5017.5854 Validation Accuracy: 0.511719
Epoch  1, Batch 126 -Loss:  3721.7642 Validation Accuracy: 0.500000
Epoch  1, Batch 127 -Loss:  3928.7798 Validation Accuracy: 0.503906
Epoch  1, Batch 128 -Loss:  4267.0054 Validation Accuracy: 0.496094
Epoch  1, Batch 129 -Loss:  4470.5010 Validation Accuracy: 0.507812
Epoch  1, Batch 130 -Loss:  3951.2195 Validation Accuracy: 0.507812
Epoch  1, Batch 131 -Loss:  3435.8967 Validation Accuracy: 0.511719
Epoch  1, Batch 132 -Loss:  4325.7578 Validation Accuracy: 0.503906
Epoch  1, Batch 133 -Loss:  3298.9395 Validation Accuracy: 0.523438
Epoch  1, Batch 134 -Loss:  5553.7856 Validation Accuracy: 0.515625
Epoch  1, Batch 135 -Loss:  4887.1890 Validation Accuracy: 0.515625
Epoch  1, Batch 136 -Loss:  5027.2969 Validation

Epoch  1, Batch 243 -Loss:  2912.0327 Validation Accuracy: 0.628906
Epoch  1, Batch 244 -Loss:  3089.6245 Validation Accuracy: 0.628906
Epoch  1, Batch 245 -Loss:  2397.3564 Validation Accuracy: 0.632812
Epoch  1, Batch 246 -Loss:  2498.1567 Validation Accuracy: 0.640625
Epoch  1, Batch 247 -Loss:  3335.8274 Validation Accuracy: 0.640625
Epoch  1, Batch 248 -Loss:  2026.6993 Validation Accuracy: 0.628906
Epoch  1, Batch 249 -Loss:  2565.2058 Validation Accuracy: 0.640625
Epoch  1, Batch 250 -Loss:  1979.4219 Validation Accuracy: 0.636719
Epoch  1, Batch 251 -Loss:  1959.5427 Validation Accuracy: 0.640625
Epoch  1, Batch 252 -Loss:  2696.1824 Validation Accuracy: 0.640625
Epoch  1, Batch 253 -Loss:  2224.0439 Validation Accuracy: 0.644531
Epoch  1, Batch 254 -Loss:  2877.4961 Validation Accuracy: 0.648438
Epoch  1, Batch 255 -Loss:  2282.2754 Validation Accuracy: 0.648438
Epoch  1, Batch 256 -Loss:  2402.9917 Validation Accuracy: 0.652344
Epoch  1, Batch 257 -Loss:  2091.4316 Validation

Epoch  1, Batch 364 -Loss:  1480.0073 Validation Accuracy: 0.667969
Epoch  1, Batch 365 -Loss:  1690.6310 Validation Accuracy: 0.664062
Epoch  1, Batch 366 -Loss:  1394.9912 Validation Accuracy: 0.660156
Epoch  1, Batch 367 -Loss:  1408.9648 Validation Accuracy: 0.660156
Epoch  1, Batch 368 -Loss:  1592.8684 Validation Accuracy: 0.656250
Epoch  1, Batch 369 -Loss:  2573.4883 Validation Accuracy: 0.656250
Epoch  1, Batch 370 -Loss:  1616.1733 Validation Accuracy: 0.660156
Epoch  1, Batch 371 -Loss:  1210.3383 Validation Accuracy: 0.660156
Epoch  1, Batch 372 -Loss:  1028.2744 Validation Accuracy: 0.660156
Epoch  1, Batch 373 -Loss:  1854.4971 Validation Accuracy: 0.660156
Epoch  1, Batch 374 -Loss:  2125.7109 Validation Accuracy: 0.675781
Epoch  1, Batch 375 -Loss:  1986.1477 Validation Accuracy: 0.667969
Epoch  1, Batch 376 -Loss:  1882.1650 Validation Accuracy: 0.671875
Epoch  1, Batch 377 -Loss:  1272.9036 Validation Accuracy: 0.664062
Epoch  1, Batch 378 -Loss:  1615.1322 Validation

Epoch  2, Batch  56 -Loss:   988.6170 Validation Accuracy: 0.699219
Epoch  2, Batch  57 -Loss:   964.9001 Validation Accuracy: 0.691406
Epoch  2, Batch  58 -Loss:   997.7360 Validation Accuracy: 0.699219
Epoch  2, Batch  59 -Loss:  1065.9978 Validation Accuracy: 0.699219
Epoch  2, Batch  60 -Loss:  1201.0833 Validation Accuracy: 0.695312
Epoch  2, Batch  61 -Loss:  1222.3116 Validation Accuracy: 0.691406
Epoch  2, Batch  62 -Loss:   982.6027 Validation Accuracy: 0.687500
Epoch  2, Batch  63 -Loss:  1080.2485 Validation Accuracy: 0.695312
Epoch  2, Batch  64 -Loss:  1121.8175 Validation Accuracy: 0.695312
Epoch  2, Batch  65 -Loss:  1139.6031 Validation Accuracy: 0.695312
Epoch  2, Batch  66 -Loss:  1323.7814 Validation Accuracy: 0.695312
Epoch  2, Batch  67 -Loss:  1469.7784 Validation Accuracy: 0.695312
Epoch  2, Batch  68 -Loss:  1384.3384 Validation Accuracy: 0.699219
Epoch  2, Batch  69 -Loss:  1128.9685 Validation Accuracy: 0.699219
Epoch  2, Batch  70 -Loss:  1387.6748 Validation

...

Epoch  9, Batch 320 -Loss:   146.7991 Validation Accuracy: 0.683594
Epoch  9, Batch 321 -Loss:   143.4625 Validation Accuracy: 0.679688
Epoch  9, Batch 322 -Loss:   159.8792 Validation Accuracy: 0.683594
Epoch  9, Batch 323 -Loss:   210.2173 Validation Accuracy: 0.671875
Epoch  9, Batch 324 -Loss:   148.4468 Validation Accuracy: 0.683594
Epoch  9, Batch 325 -Loss:   173.6471 Validation Accuracy: 0.675781
Epoch  9, Batch 326 -Loss:   198.3879 Validation Accuracy: 0.675781
Epoch  9, Batch 327 -Loss:   173.5016 Validation Accuracy: 0.691406
Epoch  9, Batch 328 -Loss:   148.4629 Validation Accuracy: 0.683594
Epoch  9, Batch 329 -Loss:   150.4878 Validation Accuracy: 0.683594
Epoch  9, Batch 330 -Loss:   217.0920 Validation Accuracy: 0.675781
Epoch  9, Batch 331 -Loss:   219.8907 Validation Accuracy: 0.687500
Epoch  9, Batch 332 -Loss:   170.6994 Validation Accuracy: 0.695312
Epoch  9, Batch 333 -Loss:   217.1252 Validation Accuracy: 0.699219
Epoch  9, Batch 334 -Loss:   145.3486 Validation

Epoch 10, Batch  12 -Loss:   164.3271 Validation Accuracy: 0.675781
Epoch 10, Batch  13 -Loss:   256.4167 Validation Accuracy: 0.667969
Epoch 10, Batch  14 -Loss:   164.6152 Validation Accuracy: 0.679688
Epoch 10, Batch  15 -Loss:   141.5237 Validation Accuracy: 0.667969
Epoch 10, Batch  16 -Loss:   201.2396 Validation Accuracy: 0.664062
Epoch 10, Batch  17 -Loss:   224.6391 Validation Accuracy: 0.660156
Epoch 10, Batch  18 -Loss:   244.1603 Validation Accuracy: 0.675781
Epoch 10, Batch  19 -Loss:   180.4553 Validation Accuracy: 0.671875
Epoch 10, Batch  20 -Loss:   197.7745 Validation Accuracy: 0.671875
Epoch 10, Batch  21 -Loss:   134.7978 Validation Accuracy: 0.667969
Epoch 10, Batch  22 -Loss:   156.4970 Validation Accuracy: 0.687500
Epoch 10, Batch  23 -Loss:   173.3336 Validation Accuracy: 0.675781
Epoch 10, Batch  24 -Loss:   177.7553 Validation Accuracy: 0.687500
Epoch 10, Batch  25 -Loss:   234.2603 Validation Accuracy: 0.703125
Epoch 10, Batch  26 -Loss:   201.4242 Validation

Epoch 10, Batch 133 -Loss:   177.5250 Validation Accuracy: 0.691406
Epoch 10, Batch 134 -Loss:   149.4355 Validation Accuracy: 0.675781
Epoch 10, Batch 135 -Loss:   164.2231 Validation Accuracy: 0.703125
Epoch 10, Batch 136 -Loss:   125.3304 Validation Accuracy: 0.710938
Epoch 10, Batch 137 -Loss:   124.8407 Validation Accuracy: 0.710938
Epoch 10, Batch 138 -Loss:   153.3304 Validation Accuracy: 0.714844
Epoch 10, Batch 139 -Loss:   169.6902 Validation Accuracy: 0.707031
Epoch 10, Batch 140 -Loss:   125.2596 Validation Accuracy: 0.687500
Epoch 10, Batch 141 -Loss:   160.3326 Validation Accuracy: 0.691406
Epoch 10, Batch 142 -Loss:   142.1692 Validation Accuracy: 0.714844
Epoch 10, Batch 143 -Loss:   198.5293 Validation Accuracy: 0.714844
Epoch 10, Batch 144 -Loss:   130.5517 Validation Accuracy: 0.703125
Epoch 10, Batch 145 -Loss:   138.4605 Validation Accuracy: 0.703125
Epoch 10, Batch 146 -Loss:   163.0799 Validation Accuracy: 0.695312
Epoch 10, Batch 147 -Loss:   151.8786 Validation

Epoch 10, Batch 254 -Loss:   149.7490 Validation Accuracy: 0.675781
Epoch 10, Batch 255 -Loss:   145.5935 Validation Accuracy: 0.675781
Epoch 10, Batch 256 -Loss:   152.8712 Validation Accuracy: 0.671875
Epoch 10, Batch 257 -Loss:   147.6519 Validation Accuracy: 0.671875
Epoch 10, Batch 258 -Loss:   181.7202 Validation Accuracy: 0.667969
Epoch 10, Batch 259 -Loss:   229.4570 Validation Accuracy: 0.671875
Epoch 10, Batch 260 -Loss:   174.2127 Validation Accuracy: 0.679688
Epoch 10, Batch 261 -Loss:   160.4003 Validation Accuracy: 0.679688
Epoch 10, Batch 262 -Loss:   139.2109 Validation Accuracy: 0.675781
Epoch 10, Batch 263 -Loss:   178.2879 Validation Accuracy: 0.699219
Epoch 10, Batch 264 -Loss:   249.2800 Validation Accuracy: 0.687500
Epoch 10, Batch 265 -Loss:   200.4019 Validation Accuracy: 0.683594
Epoch 10, Batch 266 -Loss:   222.6732 Validation Accuracy: 0.675781
Epoch 10, Batch 267 -Loss:   204.2673 Validation Accuracy: 0.687500
Epoch 10, Batch 268 -Loss:   117.7016 Validation

Epoch 10, Batch 375 -Loss:   292.3680 Validation Accuracy: 0.699219
Epoch 10, Batch 376 -Loss:   124.0011 Validation Accuracy: 0.710938
Epoch 10, Batch 377 -Loss:   130.0861 Validation Accuracy: 0.691406
Epoch 10, Batch 378 -Loss:   153.8260 Validation Accuracy: 0.691406
Epoch 10, Batch 379 -Loss:   180.3829 Validation Accuracy: 0.707031
Epoch 10, Batch 380 -Loss:   204.5737 Validation Accuracy: 0.710938
Epoch 10, Batch 381 -Loss:   169.6121 Validation Accuracy: 0.714844
Epoch 10, Batch 382 -Loss:   201.5806 Validation Accuracy: 0.703125
Epoch 10, Batch 383 -Loss:   160.9291 Validation Accuracy: 0.710938
Epoch 10, Batch 384 -Loss:   225.0729 Validation Accuracy: 0.710938
Epoch 10, Batch 385 -Loss:   175.7283 Validation Accuracy: 0.699219
Epoch 10, Batch 386 -Loss:   129.6507 Validation Accuracy: 0.710938
Epoch 10, Batch 387 -Loss:   152.8324 Validation Accuracy: 0.703125
Epoch 10, Batch 388 -Loss:   206.9316 Validation Accuracy: 0.691406
Epoch 10, Batch 389 -Loss:   197.4287 Validation